In [79]:
import re
import pandas as pd
import numpy as np

In [80]:
pd.options.mode.chained_assignment = None

In [81]:
df = pd.read_csv('input/attacks.csv', encoding='cp1252')

In [82]:
df.sample(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
2795,1986.12.11,11-Dec-1986,1986.0,Unprovoked,USA,Florida,"Ponce Inlet, New Smyrna Beach, Volusia County",Surfing,Bob Earnhardt,M,...,1.8 m to 2.1 m [6' to 7'] spinner or blacktip ...,"Orlando Sentinel, 12/12/1986, p.A1",1986.12.11-Earnhardt.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1986.12.11,1986.12.11,3508.0,NaN,NaN
9123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5092,1929.00.00.b,1929,1929.0,Unprovoked,AUSTRALIA,New South Wales,"Garden Island, Sydney",NaN,William Luckie,M,...,NaN,"G.P. Whitley, p.261",1929.00.00.b-Luckie.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1929.00.00.b,1929.00.00.b,1211.0,NaN,NaN
7637,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1739,2003.09.21,21-Sep-2003,2003.0,Unprovoked,USA,Florida,"Ponce Inlet, New Smyrna Beach, Volusia County",Surfing,Jimmy Arnold,M,...,NaN,"S. Petersohn, GSAF; Daytona Beach News Journal...",2003.09.21-Jimmy-Arnold.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2003.09.21,2003.09.21,4564.0,NaN,NaN
16706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24160,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
df.drop(["Year", "Time", "Species ", 
         "Investigator or Source", "pdf", 
         "href formula", "href", "Case Number.1", 
         "Case Number.2", "original order", "Unnamed: 22", 
         "Unnamed: 23"], axis=1, inplace=True)

In [84]:
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
9580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4692,1942.11.25,25-Nov-1942,Unprovoked,SOUTH AFRICA,KwaZulu-Natal,Umkomaas,Swimming,E.W. Bilton,M,38,"Calf bitten, leg surgically amputated",N
23517,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1686,2004.03.28,28-Mar-2004,Unprovoked,USA,Florida,"Pelican Beach Park, Satellite Beach, Brevard C...",Surfing,male,M,NaN,Heel bitten,N
2785,1987.03.30.b,30-Mar-1987,Unprovoked,AUSTRALIA,Western Australia,Fourth Beach / Twilgth Beach,Surfing,Grantley Butcher,M,18,Calf lacerated,N
18616,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [85]:
df = df[~df.isin([0, np.NAN, '', "NaN", "xx", "0"]).all(1)]

In [86]:
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
319,2016.01.02,02-Jan-2016,Unprovoked,AUSTRALIA,Queensland,Miall Island,Spearfishing,Allan Countryman,M,31,Lacerations to arms & leg,N
4693,1942.11.16,16-Nov-1942,Sea Disaster,SOLOMON ISLANDS,NaN,Battle of Guadalcanal,Thrown from destroyer when shell hit,NaN,M,19,Hip bitten,N
4386,1953.02.18,18-Feb-1953,Provoked,USA,Hawaii,"Barber’s Point, O'ahu",Bitten while cutting shark from net,James S. Takeuchi,M,NaN,Hand bitten PROVOKED INCIDENT,N
4729,1942.00.00.b,1942,Boating,NaN,NaN,NaN,"Days before the surrender of Singapore, the 3 ...","Bombardier J. Hall, Private Green of the Sherw...",M,NaN,No injury to occupants. Sharks continually fol...,N
5178,1924.10.18,18-Oct-1924,Unprovoked,AUSTRALIA,Victoria,Port Melbourne,Bathing,Fred White,M,NaN,Leg bitten,N
1833,2002.08.28,28-Aug-2002,Unprovoked,USA,Hawaii,"Kewalo Basin Channel, O'ahu",Surfing,Shawn Farden,M,16,Left foot lacerated,N
3702,1964.02.00.b,Feb-1964,Unprovoked,USA,Florida,Palm Beach County,Floating,Noel Feustel,M,12,No injury,N
3487,1968.02.00,Feb-1968,Unprovoked,MOZAMBIQUE,NaN,NaN,Diving,Sergio Peres,NaN,24,No injury,N
5958,1863.01.10,10-Jan-1863,Unprovoked,AUSTRALIA,New South Wales,"""Bellynahinch"" on the Manning River",Bathing,James Brown,M,17,FATAL,Y
5184,1924.04.22,22-Apr-1924,Unprovoked,PANAMA,150 miles offshore,NaN,"Floating, after falling or jumping off the Sta...",Claremont L. Staden,M,21,"Reported to have had ""2 fights with sharks"" be...",N


In [87]:
df = df.drop_duplicates()
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [88]:
counter = 0
df2 = pd.DataFrame()
indexes = df.index.values.tolist()
for i in range(len(indexes)-1):
    data = df.iloc[[indexes[i]]]
    for column in df.columns:
            if df.at[indexes[i], column] == df.at[indexes[i+1], column]:
                counter += 1
    if counter < 9:
        df2 = pd.concat([df2, data])
    counter = 0
df = df2

In [89]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [90]:
def surfing(x):
    if re.search("[sS]urf", str(x)):
        return "Surfing"
    else:
        return x

df["Activity"] = df["Activity"].apply(surfing)

In [91]:
def leg(x):
    if re.search("\s?([lL][eE][gG][sS]?|[fF][eEoO][eEoO][tT]?|[tT][hH][iI][gG][hH][sS]?|[aA][nN][kK][lL][eE][sS]?)\s?", str(x)):
        return "Y"
    else:
        return "N"
df["Leg Injured"] = df["Injury"].apply(leg)

In [92]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Leg Injured
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,Y
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,Y
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,Y
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,N
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,N
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,N
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,N


In [93]:
df = df[(df['Injury'].notna()) & (df['Activity'].notna())]

In [94]:
def age(x):
    if re.findall("\d\d", str(x)):
        return re.findall("\d\d", str(x))[0]
    else:
        return str(x)

df["Age"] = df["Age"].apply(age)

In [96]:
df["Age"].unique()

array(['57', '11', '48', 'nan', '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '13', '42', '3', '8', '50',
       '16', '82', '73', '68', '51', '39', '58', '47', '61', '65', '36',
       '66', '43', '9', '72', '59', '6', '27', '64', '23', '71', '44',
       '62', '63', '70', '53', 'Teen', 'teen', '77', '74', '5', '86',
       'Teens', '84', '\xa0 ', ' ', '6½', '75', '87', '67', 'Elderly',
       'adult', '9 months', '1', '(adult)', 'X', 'MAKE LINE GREEN', '81',
       '"young"', '7 or 8', 'F', 'young', '  ', '78', 'A.M.', '2½'],
      dtype=object)

In [97]:
df = df.rename(columns={'Sex ': 'Sex'})

In [98]:
df.drop(df[(df["Sex"]=="lli") | (df["Sex"]=="N") | (df["Sex"]==".")].index, inplace = True)

In [99]:
def male(x):
    if re.search("M\s", str(x)):
        return "M"
    else:
        return x

df["Sex"] = df["Sex"].apply(male)

In [100]:
df["Sex"].unique()

array(['F', 'M', nan], dtype=object)

In [101]:
df.to_csv("output/cleaned_data.csv")